In [1]:
import cv2
import mediapipe as mp
import numpy as np
import torch
from ultralytics import YOLO
import supervision as sv
import winsound
import serial

In [2]:
frequency = 2500
duration = 1500

In [3]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [4]:
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

In [5]:
path = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


In [6]:
from tensorflow.keras.models import load_model
model1 = load_model('prediction.h5')

In [11]:
S = serial.Serial('com5')

In [16]:
cap = cv2.VideoCapture(0)
model = YOLO("best.pt")
box_annotator = sv.BoxAnnotator(thickness=2,text_thickness=2,text_scale=1)
counter = 0
global text
global status
while cap.isOpened():
    Predictions = 99999
    success, image = cap.read()
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = face_mesh.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    h, w, c = image.shape
    face_3d = []
    face_2d = []
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for idx, im in enumerate(face_landmarks.landmark):
                if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                    if idx == 1:
                        nose_2d = (im.x * w, im.y * h)
                        nose_3d = (im.x * w, im.y * h, im.z * 3000)
                    x, y = int(im.x * w), int(im.y * h)
                    # Get the 2D Coordinates
                    face_2d.append([x, y])
                    # Get the 3D Coordinates
                    face_3d.append([x, y, im.z])       
            # Convert it to the NumPy array
            face_2d = np.array(face_2d, dtype=np.float64)
            # Convert it to the NumPy array
            face_3d = np.array(face_3d, dtype=np.float64)
            # The camera matrix
            focal_length = 1 * w
            cam_matrix = np.array([ [focal_length, 0, h / 2],
                                    [0, focal_length, w / 2],
                                    [0, 0, 1]])

            # The distortion parameters
            dist_matrix = np.zeros((4, 1), dtype=np.float64)

            # Solve PnP
            success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)

            # Get rotational matrix
            rmat, jac = cv2.Rodrigues(rot_vec)

            # Get angles
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)

            # Get the y rotation degree
            x = angles[0] * 360
            y = angles[1] * 360
            z = angles[2] * 360
          

            # See where the user's head tilting
            if y < -15:
                text = "Looking Left"
            elif y > 15:
                text = "Looking Right"
            elif x < -10:
                text = "Looking Down"
            elif x > 10:
                text = "Looking Up"
            else:
                text = "Forward"

            # Display the nose direction
            nose_3d_projection, jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)

            p1 = (int(nose_2d[0]), int(nose_2d[1]))
            p2 = (int(nose_2d[0] + y * 10) , int(nose_2d[1] - x * 10))            
            cv2.line(image, p1, p2, (255, 0, 0), 3)
            #cv2.putText(image, text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
    for x,y,w,h in eyes:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = image[y:y+h, x:x+w]
        eyess = eye_cascade.detectMultiScale(roi_gray)
        if len(eyess) == 0:
            print("Eyes are not detected")
            status = "Closed Eyes"
        else:
            for (ex, ey, ew, eh) in eyess:
                eyes_roi = roi_color[ey: ey+eh, ex: ex+ew]
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            if(faceCascade.empty()==False):
                print("detected")
            
            font = cv2.FONT_HERSHEY_SIMPLEX
            final_image = cv2.resize(eyes_roi, (150,150))
            final_image = np.expand_dims(final_image, axis=0)
            final_image = final_image/255.0
            Predictions = np.argmax(model1.predict(final_image))
            print(Predictions)
            if (Predictions==1):
                status = "Open Eyes"
                x1,y1,w1,h1 = 0,0,175,75
            else:
                status = "Close Eyes"
            cv2.putText(image, status, (x1 + int(w1/10),y1 + int(h1/2)), font, 1, (0, 255, 0),2)
    result = model(image)[0]
    detections = sv.Detections.from_yolov8(result)
    labels = [f"{model.model.names[class_id]} {confidence:0.1f}" for _, confidence, class_id, _ in detections]
    image = box_annotator.annotate(scene=image, detections=detections, labels=labels)
    c  = []
    for i in labels:
        a = i.split()
        c.append(a[0])
    if (Predictions==0) or ('Drowsy' in c) or (text !='Forward'):
        counter = counter + 1
    if counter > 10:
        S.write(b'1')
        counter = 0

    print(counter)
    font = cv2.FONT_HERSHEY_SIMPLEX
    x1,y1,w1,h1 = 0,0,175,75
    cv2.putText(image, text, (x1 + int(w1/10),y1 + int(h1)), font, 1, (0, 255, 0),2)
    
    cv2.imshow('OUTPUT', image)

    if cv2.waitKey(5) & 0xFF == 27:
        cv2.destroyAllWindows()
        cap.release()
        break





0: 480x640 2 Drowsys, 118.7ms
Speed: 1.0ms preprocess, 118.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


1
detected
1/1 [==============================] - 0s 24ms/step


1
Eyes are not detected


0: 480x640 3 Drowsys, 130.2ms
Speed: 1.0ms preprocess, 130.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


2


...

3



0: 480x640 1 Phone, 118.2ms
Speed: 0.0ms preprocess, 118.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Eyes are not detected
Eyes are not detected
Eyes are not detected
3



0: 480x640 2 Drowsys, 117.7ms
Speed: 1.0ms preprocess, 117.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Eyes are not detected
Eyes are not detected
4


In [15]:
cv2.destroyAllWindows()